In [2]:
import os 
import speech_recognition as sr 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment  
from transformers import pipeline
import spacy
import moviepy.editor as mp 
from moviepy.editor import VideoFileClip  
from googletrans import Translator
from gensim.summarization import summarize
from openai import OpenAI
import math
import re
import subprocess
import string
from dotenv import load_dotenv
import pysrt
import torch

os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.set_default_device("cuda")
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
m = torch.device(torch._C._get_default_device())
m

device(type='cpu')

In [6]:
torch.cuda.set_device(0) if torch.cuda.is_available() else torch.device('cpu')


  Using cached vllm-0.3.3.tar.gz (315 kB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      C:\Users\haris\AppData\Local\Temp\pip-build-env-zvtohyqk\overlay\Lib\site-packages\torch\nn\modules\transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
        device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),
      No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2'
      Traceback (most recent call last):
        File "c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['retu

In [25]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Generate text based on a prompt
prompt = "Once upon a time"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [21]:
def convert_srt_to_string(srt_file_path): 
    subs = pysrt.open(srt_file_path) 
    converted_content = '' 
    finstr = ''
    for sub in subs:  
        finstr += sub.text + ' '
        start_time = sub.start.to_time().strftime("%H:%M:%S,%f")[:-3]
        end_time = sub.end.to_time().strftime("%H:%M:%S,%f")[:-3] 
        converted_content += f"\n<strong>[{start_time} --> {end_time}]</strong>: {sub.text}\n"

    return converted_content.strip(), finstr

In [28]:
app = Flask(__name__)
CORS(app, origins="http://localhost:3000", supports_credentials=True)

nlp = spacy.load("en_core_web_sm")

@app.route('/api/analyze-audio/<string:text>', methods=['POST'])
def analyze_text(text):
    if len(text) == 0:
        return jsonify({'error': 'No text provided'}), 400
    
    doc = nlp(text) 
    person_count = len([ent.text for ent in doc.ents if ent.label_ == "PERSON"]) 
    topics = [token.text for token in doc if token.is_alpha and not token.is_stop]

    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nPerson Count: {}\n".format(str(person_count + 1)))
   
    return jsonify({'person_count' : person_count + 1 , 'topic' : topics[0]})

@app.route('/api/convert-video-to-mp3', methods=['POST'])
def convert_video_to_mp3():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    video_file = request.files['file']
    name = request.form['name']
    name = name.strip()
    name = name.replace(" ", "_")

    if video_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if video_file: 
        try:  
            print(video_file.filename)
            directory_path = os.path.join('uploads', name)
            os.makedirs(directory_path, exist_ok=True) 
            video_path = os.path.join(directory_path, video_file.filename) 
            filesname, extension = os.path.splitext(video_file.filename)
            audio_path = os.path.join(directory_path, "{}.mp3".format(filesname)) 
            video_file.save(video_path)
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(audio_path)
            audio_clip.close()
            video_clip.close()

            return jsonify({'message': 'Video converted to MP3 successfully', 'audio_file': audio_path}), 200
        except Exception as e:
            return jsonify({'error': 'Conversion error', 'details': str(e)}), 500

@app.route('/api/convert-mp3-to-text', methods=['POST'])
def convert_mp3_to_text():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file'] 
    name = request.form['name']
     
    name = name.strip()
    name = name.replace(" ", "_")

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:  
        filesname, extension = os.path.splitext(file.filename) 
        filesname = filesname.strip()
        filesname = filesname.replace(" ", "_")

        full_filename = filesname+extension
        directory_path = os.path.join('uploads', name)
        os.makedirs(directory_path, exist_ok=True) 
        mp3_filename = os.path.join(directory_path, full_filename)

        file.save(mp3_filename) 
        segment_dir = 'segments'  
        os.makedirs(segment_dir, exist_ok=True)  
        
        #-----------------------------------------------------------------------------------------
        command1 = f"ffmpeg -i ./uploads/{name}/{filesname}.mp3 -ar 16000 -ac 1 -c:a pcm_s16le ./uploads/{name}/final.wav"  
        command2 = f".\\WHISPER_EXE_FILES\\main.exe -f ./uploads/{name}/final.wav -m .\\WHISPER_EXE_FILES\\ggml-base.en.bin -osrt -t 8 --language auto" 
        
        result = subprocess.run(command1, shell=True, capture_output=True, text=True)
        result2 = subprocess.run(command2, shell=True, capture_output=True, text=True)  

        srt_file_path = f'./uploads/{name}/final.wav.srt'
        text_path = os.path.join(directory_path, "{}.txt".format(filesname)) 
        result_string, joined_text = convert_srt_to_string(srt_file_path)
        os.remove(f"./uploads/{name}/final.wav")

        result_string = result_string.replace("\n", "<br>")

        with open(text_path, 'w') as fil:
            fil.write("\nConverted Text: {}\n".format(str(joined_text)))
 
        return jsonify({'text': joined_text, 'whisper_str': result_string})

@app.route('/api/convert-mp4-to-text', methods=['POST'])
def convert_mp4_to_text():
    language = request.form['language']
    file = request.files['file']
    name = request.form['name']
    
    name = name.strip()
    name = name.replace(" ", "_")

    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")

    file = os.path.abspath('./uploads/{}/{}.mp3'.format(name, filesname)) 
    directory_path = os.path.join('uploads', name)
    os.makedirs(directory_path, exist_ok=True)
    if file: 
        mp3_filename = file   
        command1 = f"ffmpeg -i ./uploads/{name}/{filesname}.mp3 -ar 16000 -ac 1 -c:a pcm_s16le ./uploads/{name}/final.wav"  
        result = subprocess.run(command1, shell=True, capture_output=True, text=True)

    segment_dir = 'segments'
    try:
        if not os.path.exists(segment_dir):
            os.mkdir(segment_dir)
    except Exception as e:
        print(f"Error creating directory: {e}")


    command2 = f".\\WHISPER_EXE_FILES\\main.exe -f ./uploads/{name}/final.wav -m .\\WHISPER_EXE_FILES\\ggml-base.en.bin -osrt -t 8 --language auto" 
    result2 = subprocess.run(command2, shell=True, capture_output=True, text=True)  

    srt_file_path = f'./uploads/{name}/final.wav.srt'
    text_path = os.path.join(directory_path, "{}.txt".format(filesname)) 
    result_string, joined_text = convert_srt_to_string(srt_file_path)
    os.remove(f"./uploads/{name}/final.wav")
    result_string = result_string.replace("\n", "<br>")

    with open(text_path, 'w') as fil:
        fil.write("\nConverted Text: {}\n".format(str(joined_text)))

    return jsonify({'text': joined_text, 'whisper_str': result_string})

@app.route('/api/translate_toar/<string:text>', methods=['GET'], endpoint='translate_to_ar')
def translate_to_ar(text):    
    translator = Translator() 
    arabic_translation = translator.translate(text,  src='auto', dest='ar').text
    return jsonify({'translated_txt': arabic_translation})

@app.route('/api/translate_totr/<string:text>', methods=['GET'], endpoint='translate_to_tr')
def translate_to_tr(text):  
    translator = Translator()    

    turkish_translation = translator.translate(text, src='auto', dest='tr').text
    return jsonify({'translated_txt': turkish_translation})

@app.route('/api/translate_toen/<string:text>', methods=['GET'], endpoint='translate_to_en')
def translate_to_en(text):  
    translator = Translator()    

    english_translation = translator.translate(text, src='auto', dest='en').text
    return jsonify({'translated_txt': english_translation})

@app.route('/api/translate_tohi/<string:text>', methods=['GET'], endpoint='translate_to_hi')
def translate_to_hi(text):  
    translator = Translator()    

    hindi_translation = translator.translate(text, src='auto', dest='hi').text
    return jsonify({'translated_txt': hindi_translation})

def add_newlines_every_n_words(input_string, n=10):
    words = input_string.split()
    output_string = ''
    for i, word in enumerate(words):
        if i > 0 and i % n == 0:
            output_string += '\n'
        output_string += word + ' '
    return output_string.strip()

@app.route('/api/findtopic/<string:text>', methods=['POST'])
def topic_finder(text):
    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    pipe = pipeline("text-classification", model="unitary/toxic-bert")
    pipe2 = pipeline("summarization", model="google/pegasus-xsum")

    topic = pipe(text)  
    topic = topic[0]['label']

    topic2 = pipe2(text, max_length = 20)
    topic2 = topic2[0]['summary_text']

    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nTopic: {}\n".format(topic2))
    
    return jsonify({'topic': topic, 'topic2': topic2})

@app.route('/api/findSummary', methods=['POST'])
def summary_find():
    text = request.form['text']
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    
    temp = add_newlines_every_n_words(text, 30)
    try: 
        summary = summarize(temp)
        print("len: ", len(summary))
    except:
        print("Error in summarization using gensim") 
        summary = ""

    if len(summary) == 0:
        output = pipe(text) 
        summary = output[0]['summary_text']
             
    translator = Translator() 
    arabic_summary = translator.translate(summary, src='en', dest='ar').text
    turkish_summary = translator.translate(summary, src='en', dest='tr').text

    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nSummary: {}\n".format(str(summary)))

    return jsonify({'summary_en': str(summary), 'summary_ar': arabic_summary, 'summary_tr': turkish_summary})

@app.route('/api/sentiment/<string:text>', methods=['POST'])
def sentiment(text):
    name = request.form['name']
    file = request.files['file']

    name = name.strip()
    name = name.replace(" ", "_")

    nltk.download('vader_lexicon')
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    positive_percent = round(scores['pos'] * 100, 2)
    negative_percent = round(scores['neg'] * 100, 2)

    total = positive_percent + negative_percent
    if (total == 0):
        fin_pos = 50
        fin_neg = 50
    else:
        fin_pos = round((positive_percent/total)*100, 2)
        fin_neg = round((negative_percent/total)*100, 2) 


    directory_path = os.path.join('uploads', name) 
    filesname, extension = os.path.splitext(file.filename)
    filesname = filesname.strip()
    filesname = filesname.replace(" ", "_")
    text_path = os.path.join(directory_path, "{}.txt".format(filesname))  

    with open(text_path, 'a') as fil:
        fil.write("\nPositive Sentiment: {}% \nNegative Sentiment: {}%".format(fin_pos, fin_neg))

    return jsonify({'positive': fin_pos, 'negative': fin_neg})

if __name__ == '__main__': 
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


hindi.mp4
MoviePy - Writing audio in uploads\Haris_Rehman\hindi.mp3


127.0.0.1 - - [19/Apr/2024 15:05:47] "POST /api/convert-video-to-mp3 HTTP/1.1" 200 -


MoviePy - Done.


127.0.0.1 - - [19/Apr/2024 15:05:54] "POST /api/convert-mp4-to-text HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2024 15:05:56] "GET /api/translate_toen/%20Namaskar,%20Mira%20Nam%20language%20in%20Pei,%20%20or%20May%20hyper%20polyglot%20gigachad%20alpha%20mel%20hum.%20%20Miri%20Umar%20Chobes%20Salhe,%20or%20May%20America%20Sei%20hum.%20%20May%20Hindissi%20Krahahum,%20%20Kyungke%20Moojilak%20Thaheki,%20%20Modisab%20Sei%20Bahut,%20Sei%20Bahut,%20Bahut,%20Sundarhe.%20%20Or%20May%20Baharet,%20Jan%20Achaathahum,%20%20or%20May, HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2024 15:05:57] "POST /api/analyze-audio/Namaskar,%20mira%20nam%20language%20in%20pei,%20or%20may%20hyper%20polyglot%20gigachad%20alpha%20mel%20hum.Miri%20Umar%20Chobes%20Salhe,%20or%20May%20America%20Sei%20Hum.May%20Hindissi%20Krahhum,%20Kyungke%20Moojilak%20Thaheki,%20Modisab%20Sei%20Bahut,%20Sei%20Bahut,%20Bahut,%20Sundarhe.Or%20may%20baharat,%20jan%20achaathahum,%20or%20may, HTTP/1.1" 200 -


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
127.0.0.1 - - [19/Apr/2024 15:06:10] "POST /api/findtopic/Namaskar,%20mira%20nam%20language%20in%20pei,%20or%20may%20hyper%20polyglot%20gigachad%20alpha%20mel%20hum.Miri%20Umar%20Chobes%20Salhe,%20or%20May%20America%20Sei%20Hum.May%20Hindissi%20Krahhum,%20Kyungke%20Moojilak%20Thaheki,%20Modisab%20Sei%20Bahut,%20Sei%20Bahut,%20Bahut,%20Sundarhe.Or%20may%20baharat,%20jan%20achaathahum,%20or%20may, HTTP/1.1" 200 -
